# データベース_ICSD_標準偏差

In [1]:
# import module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)


In [2]:
# 不要なもの削除
atomic_data = pd.read_csv("/media/sf_kyoyu/data/atomicdata処理後.csv").drop(['Z','Nf','CuKa','MoKa'], axis=1).set_index("Symbol")

# --------------単体削除
compounds_data = pd.read_csv("/media/sf_kyoyu/data/complist_integer_compounds_pickup重複なし.csv").set_index("Compounds")\
.drop(['H1','Li1','Be1',"B1","C1","N1","O1","Na1","Mg1","Al1","Si1","P1","S1","K1","Ca1","Sc1",\
                                           "Ti1","V1","Cr1","Mn1","Fe1","Co1","Ni1","Cu1","Zn1","Ga1","Ge1","As1",\
                                           "Se1","Rb1","Sr1","Y1","Zr1","Nb1","Mo1","Ru1","Rh1",\
                                           "Pd1","Ag1","Cd1","In1","Sn1","Sb1","Te1","I1","Cs1","Ba1","La1",\
                                          "Ce1","Pr1","Nd1","Sm1","Eu1","Gd1","Dy1","Ho1","Er1","Tm1",\
                                          "Yb1","Lu1","Hf1","Ta1","W1","Re1","Os1","Ir1","Pt1","Au1","Hg1","Tl1","Pb1","Bi1"])
average_data = pd.read_csv("/media/sf_kyoyu/data/average_database_ICSD_重複なし.csv").set_index("Compounds").\
drop(['Z','Nf','CuKa','MoKa'], axis=1).drop(['H1','Li1','Be1',"B1","C1","N1","O1",\
                                           "Na1","Mg1","Al1","Si1","P1","S1","K1","Ca1","Sc1",\
                                           "Ti1","V1","Cr1","Mn1","Fe1","Co1","Ni1","Cu1","Zn1","Ga1","Ge1","As1",\
                                           "Se1","Rb1","Sr1","Y1","Zr1","Nb1","Mo1","Ru1","Rh1",\
                                           "Pd1","Ag1","Cd1","In1","Sn1","Sb1","Te1","I1","Cs1","Ba1","La1",\
                                          "Ce1","Pr1","Nd1","Sm1","Eu1","Gd1","Dy1","Ho1","Er1","Tm1",\
                                          "Yb1","Lu1","Hf1","Ta1","W1","Re1","Os1","Ir1","Pt1","Au1","Hg1","Tl1","Pb1","Bi1"])

In [17]:
# 標準偏差データの作成
A = atomic_data
B = compounds_data
C = average_data

# ----------------jは化合物名、kは元素情報
sd_data = []
for j in np.array(B.index):
    row_list = []
    for k in np.array(A.columns):
        deviation = [(A.loc[i,k]-C.loc[j,k])**2 for i in np.array(A.index)]
        D = np.array(B.loc[j,:])*deviation
        E = sum(D)
        F = np.sqrt(E/sum(B.loc[j,:]))
        row_list.append(F)
    sd_data.append(row_list)
sd_data = pd.DataFrame(sd_data, index = np.array(B.index), columns = np.array(A.columns))
sd_data.to_csv("/media/sf_kyoyu/data/standard_deviation_database_ICSD.csv")

In [3]:
sd_data = pd.read_csv("/media/sf_kyoyu/data/standard_deviation_database_ICSD.csv").rename(columns = {'Unnamed: 0':'Compounds'})\
.set_index("Compounds")

In [4]:
# 標準偏差ー逆数データ
sd_data_inverse = 1/sd_data
sd_data_inverse = sd_data_inverse.replace('inf',0)

# 標準偏差ー二乗データ
sd_data_square = sd_data*sd_data

# 標準偏差ー対数データ
sd_data_log = np.log(sd_data)
sd_data_log = sd_data_log.replace('-inf',0).fillna(0)

In [5]:
# 標準偏差データの標準化
scaled_list = []
for i in np.array(sd_data.columns):
    sc = StandardScaler()
    sc.fit(sd_data.loc[:,i])
    array_scaled = sc.transform(sd_data.loc[:,i])
    scaled_list.append(array_scaled)
df = pd.DataFrame(scaled_list).T
df.index = np.array(sd_data.index)
df.columns = np.array(sd_data.columns)

# 標準偏差ー逆数データの標準化
scaled_list_inverse = []
for i in np.array(sd_data_inverse.columns):
    sc = StandardScaler()
    sc.fit(sd_data_inverse.loc[:,i])
    array_scaled_inverse = sc.transform(sd_data_inverse.loc[:,i])
    scaled_list_inverse.append(array_scaled_inverse)
df_inverse = pd.DataFrame(scaled_list_inverse).T
df_inverse.index = np.array(sd_data_inverse.index)
df_inverse.columns = np.array(sd_data_inverse.columns)

# 標準偏差ー二乗データの標準化
scaled_list_square = []
for i in np.array(sd_data_square.columns):
    sc = StandardScaler()
    sc.fit(sd_data_square.loc[:,i])
    array_scaled_square = sc.transform(sd_data_square.loc[:,i])
    scaled_list_square.append(array_scaled_square)
df_square = pd.DataFrame(scaled_list_square).T
df_square.index = np.array(sd_data_square.index)
df_square.columns = np.array(sd_data_square.columns)

# 標準偏差ー対数データの標準化
scaled_list_log = []
for i in np.array(sd_data_log.columns):
    sc = StandardScaler()
    sc.fit(sd_data_log.loc[:,i])
    array_scaled_log = sc.transform(sd_data_log.loc[:,i])
    scaled_list_log.append(array_scaled_log)
df_log = pd.DataFrame(scaled_list_log).T
df_log.index = np.array(sd_data_log.index)
df_log.columns = np.array(sd_data_log.columns)

# # 標準偏差ー全データ連結
df_all = pd.concat([df, df_inverse, df_square, df_log], axis=1)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [7]:
# 主成分分析
# ---------------標準偏差そのままデータ
pca = PCA(n_components = 3, whiten = False)
pca.fit(df)
df_pca = pca.transform(df)
E = pca.explained_variance_ratio_
pca_components = pd.DataFrame(pca.components_ ,index = ["第一主成分","第二主成分","第三主成分"], columns = np.array(df.columns))
df_pca = pd.DataFrame(df_pca, columns = ["第一主成分","第二主成分","第三主成分"], index = df.index)
print pca.components_
print np.cumsum(E)

# ----------------標準偏差ALLデータ
pca = PCA(n_components = 3, whiten = False)
pca.fit(df_all)
df_pca_all = pca.transform(df_all)
E_all = pca.explained_variance_ratio_
pca_components_all = pd.DataFrame(pca.components_ ,index = ["第一主成分","第二主成分","第三主成分"], columns = np.array(df_all.columns))
df_pca_all = pd.DataFrame(df_pca_all, columns = ["第一主成分","第二主成分","第三主成分"], index = df_all.index)
print pca.components_
print np.cumsum(E_all)

[[-0.2550511  -0.27908785 -0.06456089 -0.3321315   0.12179227 -0.07788099
  -0.39402147 -0.20297551 -0.39165414 -0.36851858 -0.37652545 -0.16724089
  -0.24373568  0.00090504 -0.08076468  0.01892096]
 [ 0.36251764 -0.2293651  -0.14855692 -0.1367327   0.13876884  0.47746364
  -0.11789193 -0.10253885 -0.06888688 -0.09718872 -0.02037716  0.33287008
   0.27438289  0.46731107  0.28468625  0.05870222]
 [-0.27674841 -0.00392374  0.03464494  0.04088885 -0.31793174 -0.38592466
  -0.07887524 -0.07209589 -0.04110776 -0.10255971 -0.15424093  0.49097824
   0.34883177 -0.19124163  0.47212356  0.02948461]]
[ 0.32948045  0.47884723  0.60745511]
[[ -1.49976975e-01  -1.70520051e-01  -4.50034164e-02  -1.99603033e-01
    6.80941259e-02  -4.79980857e-02  -2.35191809e-01  -1.23490849e-01
   -2.32629608e-01  -2.17404884e-01  -2.22633394e-01  -9.81829641e-02
   -1.46251590e-01  -5.74042018e-03  -4.47051556e-02   5.31100740e-03
    8.85744564e-02   1.00820591e-01  -8.50640094e-02   1.14152330e-01
   -4.48849404

In [10]:
# 主成分分析後の二次元プロット
plt.scatter(df_pca.ix[:,0], df_pca.ix[:,1])
plt.show()

/usr/lib/python2.7/dist-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
